In [1]:
import sys

runtype = "daily"
report_date = "2021-06-29"

no_of_cpu = 2
max_cores = 2
executor_mem = '16g'

Job_Name = 'claims_pipeline_TP'

print("JOB NAME = "+str(Job_Name))
print("No_of_cpu="+str(no_of_cpu))
print("Max_cores="+str(max_cores))
print("Executor_mem="+str(executor_mem))
print("Runtype="+str(runtype))
print("Report_date="+str(report_date))

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
import time;
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
conf = pyspark.SparkConf()
#import numpy
import calendar
import numpy as np
import datetime
from pyspark.sql.functions import *
import pyspark.sql.functions as sf
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import lit
# import simplejson as json
import json, pprint, requests

mesos_ip = 'mesos://10.33.195.18:5050'
spark.stop()
conf.setMaster(mesos_ip)
conf.set('spark.executor.cores',no_of_cpu)
conf.set('spark.cores.max',max_cores)
conf.set('spark.executor.memory',executor_mem)
conf.set("spark.app.name", Job_Name)
#conf.set('spark.memory.fraction','.2')
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")
conf.set('es.nodes.wan.only','true')
#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true"); 
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','8g')
conf.set('spark.sql.shuffle.partitions','300')
#conf.set('spark.sql.crossJoin.enabled', 'true')
# conf.set('es.nodes',es_nodes_prod)
# conf.set('es.port',es_port_prod)
#conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
#conf.set('spark.num.executors','5')

# conf.set('spark.es.net.http.auth.user','biu_dev')
# conf.set('spark.es.net.http.auth.pass','Biudev!23')

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)
import json, pprint, requests

conf = pyspark.SparkConf()
conf.set('es.http.timeout','5m')
conf.set('es.http.retries','10')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1982,None,pyspark,idle,,,✔


SparkSession available as 'spark'.
JOB NAME = claims_pipeline_TP
No_of_cpu=2
Max_cores=2
Executor_mem=16g
Runtype=daily
Report_date=2021-06-29

In [ ]:
def load_gpdb_jdbc(col_str,dbtable,port=1107,time_filter=None,col_name='REPORTEDDATE'):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                        "user": "gpanalytics",
                        "password": "ana@678",
                        "dbschema": "public",
                        "dbtable": dbtable,
                        "partitions": 2,
                        "server.port": port}
   
    if time_filter:
        data = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
                .select(col_str).drop_duplicates()
   
    return data

### FEATURE SUMMARY

#### SELECTING ONLY IMPORTANT COLUMNS FROM TABLE

In [ ]:
time_filter={'start_date':'2019-10-01','end_date':'2021-01-01'}

In [ ]:
cols = ['FEATURESUMMARYID','CLAIMNUMBER','POLICYNUMBER','CWPREASON','REPORTEDDATE','PRODUCTNAME','RPDREASON','LASTMODIFIEDON',
       'CLAIMSTATUS','SETTLINGOFFICE','EXAMINERNAME']

In [ ]:
# Loading data from green plum through py-spark : Getting claim_bpm_featuresummary claim table with feature summary from date year Jan - 2019 
feature_sum = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_featuresummary',port=1107,time_filter=time_filter)

In [ ]:
feature_sum = feature_sum.withColumn("CLAIMSTATUS", trim(col("CLAIMSTATUS")))
feature_sum = feature_sum.withColumn("CLAIMSTATUS", upper(col("CLAIMSTATUS")))

In [ ]:
feature_sum = feature_sum.filter(col('CLAIMSTATUS')=='OPEN')

In [ ]:
# Now probably we need to get only the last/recent claim date.
tmp = feature_sum
# date and time conversion 
tmp = tmp.withColumn("LASTMODIFIEDON",
                     to_timestamp(unix_timestamp(tmp["LASTMODIFIEDON"],
                                                 "yyyy-MM-dd HH:mm:ss").cast("timestamp")))

# group by descending order
window = Window.partitionBy(tmp['FEATURESUMMARYID']).orderBy(tmp['LASTMODIFIEDON'].desc())
# 
feat_sum_col = tmp.select('*', rank().over(window).alias('rank')).filter(col('rank') < 2)

In [ ]:
feat_sum_col=feat_sum_col.drop_duplicates()

In [ ]:
feat_sum_col=feat_sum_col.filter(col('POLICYNUMBER').isNotNull()).drop(feat_sum_col.rank)

In [ ]:
# feat_sum_col.count()

### CLAIM FEATURE MAPPING

#### SELECTING ONLY IMPORTANT COLUMNS FROM TABLE

In [ ]:
cols= ['FEATURESUMMARYID','COVERNAME','FEATURENUMBER']

In [ ]:
claim_fea_map_col = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_claimfeaturemapping',port=1107,time_filter=None)

In [ ]:
claim_fea_map_col = claim_fea_map_col.withColumn("COVERNAME", trim(col("COVERNAME")))
claim_fea_map_col = claim_fea_map_col.withColumn("COVERNAME", lower(col("COVERNAME")))

In [ ]:
claim_fea_map_col=claim_fea_map_col.drop_duplicates()

In [ ]:
gscPythonOptions = {
                        "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                        "user": "gpanalytics",
                        "password": "ana@678",
                        "dbschema": "analytics",
                        "dbtable": "TP_covernames",
                        "partitions": 2,
                        "server.port": 1107}
tp_covernames = sqlContext.read.format("jdbc").options(**gscPythonOptions).load().drop_duplicates()

In [ ]:
tp_covernames = tp_covernames.withColumn("corporatecd", trim(col("corporatecd")))
tp_covernames = tp_covernames.withColumn("corporatecd", lower(col("corporatecd")))

In [ ]:
tp_covernames.columns

In [ ]:
claim_fea_tp = claim_fea_map_col.join(tp_covernames.select('corporatecd'),[claim_fea_map_col.COVERNAME==tp_covernames.corporatecd],
                                      how='inner').drop_duplicates()

In [ ]:
# claim_fea_tp.count()

In [ ]:
tp_claims = feat_sum_col.join(claim_fea_tp,[feat_sum_col.FEATURESUMMARYID==claim_fea_tp.FEATURESUMMARYID],how='inner').drop(claim_fea_tp.FEATURESUMMARYID)

In [ ]:
tp_claims.columns

### LOSS DETAILS

#### SELECTING ONLY IMPORTANT COLUMNS FROM TABLE

In [ ]:
cols = ['FEATURESUMMARYID', 'LOSSTIME','STATE','CITY','DATEOFLOSS','LOSSDESCRIPTION','INJURYTYPE']

In [ ]:
loss_details_col = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_lossdetails',port=1107,time_filter=None)

In [ ]:
loss_details_col = loss_details_col.withColumnRenamed('STATE','LOSS_STATE')
loss_details_col = loss_details_col.withColumnRenamed('CITY','LOSS_CITY')

In [ ]:
loss_details_col = loss_details_col.drop_duplicates()

In [ ]:
tp_claims_1=tp_claims.join(loss_details_col,[tp_claims.FEATURESUMMARYID==loss_details_col.FEATURESUMMARYID],
                          how='left').drop(loss_details_col.FEATURESUMMARYID)

In [ ]:
# tp_claims_1.count()

### LITIGATION_DETAILS

In [ ]:
cols =['FEATURESUMMARYID','COURTNAME','COURTDISTRICT','LITIGATIONTYPE','PLAINTIFFATTORNEY','CREATEDON']

In [ ]:
litigation_details = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_litigationdetails',port=1107,time_filter=None)

In [ ]:
litigation_details.select('COURTNAME').distinct().count()

In [ ]:
litigation_details = litigation_details.withColumn("COURTNAME",sf.upper(sf.trim(sf.col("COURTNAME"))))
litigation_details = litigation_details.withColumn("COURTDISTRICT",sf.upper(sf.trim(sf.col("COURTDISTRICT"))))
litigation_details = litigation_details.withColumn("PLAINTIFFATTORNEY",sf.upper(sf.trim(sf.col("PLAINTIFFATTORNEY"))))

In [ ]:
# litigation_details.count()

In [ ]:
# Now probably we need to get only the last/recent claim date.
tmp = litigation_details
# date and time conversion 
tmp = tmp.withColumn("CREATEDON",to_timestamp(unix_timestamp(tmp["CREATEDON"],"yyyy-MM-dd HH:mm:ss").cast("timestamp")))
# group by descending order
window = Window.partitionBy(tmp['FEATURESUMMARYID']).orderBy(tmp['CREATEDON'].desc())
litigation_details = tmp.select('*', rank().over(window).alias('rank')).filter(col('rank') < 2)

In [ ]:
litigation_details=litigation_details.drop('CREATEDON')

In [ ]:
litigation_details=litigation_details.drop_duplicates()

In [ ]:
# litigation_details.count()

### COURT MASTER

In [ ]:
cols = ['COURTNAME','DISTRICT','STATE']

In [ ]:
court_master = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_courtmaster',port=1107,time_filter=None).drop_duplicates()

In [ ]:
court_master = court_master.withColumnRenamed('STATE','COURT_STATE')
court_master = court_master.withColumnRenamed('DISTRICT','COURT_DISTRICT')

In [ ]:
# court_master.select('COURTNAME').distinct().count()

In [ ]:
court_master = court_master.withColumn("COURTNAME",sf.upper(sf.trim(sf.col("COURTNAME"))))
court_master = court_master.withColumn("COURT_DISTRICT",sf.upper(sf.trim(sf.col("COURT_DISTRICT"))))

In [ ]:
court_master = court_master.drop_duplicates()

In [ ]:
# court_master.count()

#### COURT MASTER JOIN WITH LITIGATION

In [ ]:
cond=[litigation_details.COURTNAME == court_master.COURTNAME, litigation_details.COURTDISTRICT == court_master.COURT_DISTRICT]
court_details = litigation_details.join(court_master, cond, how='left').drop(court_master.COURTNAME)
# .drop(court_master.COURTNAME).drop(court_master.DISTRICT)

In [ ]:
court_details=court_details.drop_duplicates()

In [ ]:
tp_claims_2=tp_claims_1.join(court_details,[tp_claims_1.FEATURESUMMARYID==court_details.FEATURESUMMARYID],how='left').drop(court_details.FEATURESUMMARYID)

In [ ]:
tp_claims_2.columns

### POLICY DETAILS

In [ ]:
cols = ['FEATURESUMMARYID','DAT_POLICY_STARTDATE','DAT_POLICY_ENDDATE','TXT_POLICY_START_TIME','TXT_BUSINESS_TYPE',
        'TXT_PRODUCER_CD', 'TXT_PRODUCER_NAME','TXT_REG_NO','TXT_POLICY_NO_CHAR']

In [ ]:
policy_det_col = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_policydetailsaudit',port=1107,time_filter=None)

In [ ]:
policy_det_col=policy_det_col.drop_duplicates()

In [ ]:
tp_claims_3=tp_claims_2.join(policy_det_col,[tp_claims_2.POLICYNUMBER==policy_det_col.TXT_POLICY_NO_CHAR,
                                             tp_claims_2.FEATURESUMMARYID==policy_det_col.FEATURESUMMARYID],how='left').drop(policy_det_col.FEATURESUMMARYID).drop_duplicates()

In [ ]:
tp_claims_3.columns

### FIR DETAILS

In [ ]:
cols = ['FEATURESUMMARYID','CITY','STATE','DISTRICT','FIRDATE','ISINSUREDVEHICLENOTIDENTIFIEDINFIR','POLICESTATIONADDRESS1',
        'POLICESTATIONADDRESS2','POLICESTATIONADDRESS3','POLICESTATIONLOCATION','PINCODE','CREATEDON']
firdetails = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_firdetailsclaimprocessing',port=1107,time_filter=None)

In [ ]:
firdetails = firdetails.withColumnRenamed('STATE','FIR_STATE')
firdetails = firdetails.withColumnRenamed('DISTRICT','FIR_DISTRICT')
firdetails = firdetails.withColumnRenamed('CITY','FIR_CITY')

In [ ]:
firdetails = firdetails.withColumn("POLICESTATIONLOCATION",sf.upper(sf.trim(sf.col("POLICESTATIONLOCATION"))))
firdetails = firdetails.withColumn("POLICESTATIONADDRESS1",sf.upper(sf.trim(sf.col("POLICESTATIONADDRESS1"))))
firdetails = firdetails.withColumn("POLICESTATIONADDRESS2",sf.upper(sf.trim(sf.col("POLICESTATIONADDRESS2"))))
firdetails = firdetails.withColumn("POLICESTATIONADDRESS3",sf.upper(sf.trim(sf.col("POLICESTATIONADDRESS3"))))

In [ ]:
# Now probably we need to get only the last/recent claim date.
tmp = firdetails
# date and time conversion 
tmp = tmp.withColumn("CREATEDON",to_timestamp(unix_timestamp(tmp["CREATEDON"],"yyyy-MM-dd HH:mm:ss").cast("timestamp")))
# group by descending order
window = Window.partitionBy(tmp['FEATURESUMMARYID']).orderBy(tmp['CREATEDON'].desc())
firdetails = tmp.select('*', rank().over(window).alias('rank')).filter(col('rank') < 2)

In [ ]:
firdetails=firdetails.drop('rank').drop('CREATEDON')

In [ ]:
firdetails=firdetails.drop_duplicates()

In [ ]:
tp_claims_4=tp_claims_3.join(firdetails,[tp_claims_3.FEATURESUMMARYID==firdetails.FEATURESUMMARYID],how='left').drop(firdetails.FEATURESUMMARYID).drop_duplicates()

In [ ]:
tp_claims_4.columns

### HOSPITAL DETAILS

In [ ]:
cols = ['FEATURESUMMARYID','HOSPITALADDRESS','HOSPITALNAME','HOSPITALMASTERID','CREATEDON']
hosp_details = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_hospitaldetails',port=1107,time_filter=None)

In [ ]:
# Now probably we need to get only the last/recent claim date.
tmp = hosp_details
# date and time conversion 
tmp = tmp.withColumn("CREATEDON",to_timestamp(unix_timestamp(tmp["CREATEDON"],"yyyy-MM-dd HH:mm:ss").cast("timestamp")))
# group by descending order
window = Window.partitionBy(tmp['FEATURESUMMARYID']).orderBy(tmp['CREATEDON'].desc())
hosp_details = tmp.select('*', rank().over(window).alias('rank')).filter(col('rank') < 2)

In [ ]:
hosp_details=hosp_details.drop('rank').drop('CREATEDON')

In [ ]:
hosp_details=hosp_details.drop_duplicates()

In [ ]:
hosp_details = hosp_details.withColumn("HOSPITALNAME",sf.upper(sf.trim(sf.col("HOSPITALNAME"))))

#### LOADING HOSPITAL MASTER TO GET ADDRESS DETAILS

In [ ]:
cols = ['HOSPITALMASTERID', 'HOSPITALNAME','DISTRICT', 'STATE']
hosp_master = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_hospitalmaster',port=1107,time_filter=None)

In [ ]:
hosp_master = hosp_master.withColumnRenamed('STATE','HOSPITAL_STATE')
hosp_master = hosp_master.withColumnRenamed('DISTRICT','HOSPITAL_DISTRICT')

In [ ]:
hosp_master = hosp_master.drop_duplicates()

#### JOIN HOSPITAL DETAILS WITH MASTER TABLE

In [ ]:
cond=[hosp_details.HOSPITALMASTERID == hosp_master.HOSPITALMASTERID]
hosp_final = hosp_details.join(hosp_master, cond, how='left').drop(hosp_master.HOSPITALNAME).drop(hosp_master.HOSPITALMASTERID)

In [ ]:
hosp_final = hosp_final.drop_duplicates()

In [ ]:
tp_claims_5 = tp_claims_4.join(hosp_final,[tp_claims_4.FEATURESUMMARYID==hosp_final.FEATURESUMMARYID],how='left').drop(hosp_final.FEATURESUMMARYID).drop_duplicates()

In [ ]:
tp_claims_5.columns

### INJURED DECEASED DETAILS

In [ ]:
cols = ['FEATURESUMMARYID','INJUREDDECEASEDNAME','CREATEDON']
injured_details = load_gpdb_jdbc(col_str=cols,dbtable='claim_bpm_injureddeceaseddetails',port=1107,time_filter=None)

In [ ]:
# Now probably we need to get only the last/recent claim date.
tmp = injured_details
# date and time conversion 
tmp = tmp.withColumn("CREATEDON",to_timestamp(unix_timestamp(tmp["CREATEDON"],"yyyy-MM-dd HH:mm:ss").cast("timestamp")))
# group by descending order
window = Window.partitionBy(tmp['FEATURESUMMARYID']).orderBy(tmp['CREATEDON'].desc())
injured_details = tmp.select('*', rank().over(window).alias('rank')).filter(col('rank') < 2)

In [ ]:
injured_details = injured_details.drop('rank').drop('CREATEDON')

In [ ]:
injured_details = injured_details.drop_duplicates()

In [ ]:
tp_claims_6 = tp_claims_5.join(injured_details,[tp_claims_5.FEATURESUMMARYID==injured_details.FEATURESUMMARYID],
                              how='left').drop(injured_details.FEATURESUMMARYID).drop_duplicates()

In [ ]:
tp_claims_6.columns

In [ ]:
# tp_claims_6.columns

In [ ]:
tp_claims_6 = tp_claims_6.withColumn("TXT_POLICY_NO_CHAR",sf.upper(sf.trim(sf.col("TXT_POLICY_NO_CHAR"))))

In [ ]:
tp_claims_6 = tp_claims_6.withColumn("TXT_POLICY_NO_CHAR",col("TXT_POLICY_NO_CHAR").cast(StringType()))

### Premium Register

In [ ]:
# input_df_distinct = tp_claims_6.select('POLICYNUMBER').distinct()
# input_df_distinct = input_df_distinct.filter(input_df_distinct.POLICYNUMBER.isNotNull())

In [ ]:
gscPythonOptions = {
        "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpanalytics","password": "ana@678","dbschema": "registers",
        "dbtable": "premium_register","partitions":16,"server.port": '1150-1170',"partitionColumn":"row_num"}
prem_reg = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('POLICY_NO','BRANCH','BRANCH_STATE_NAME','COVERNOTENUMBER','CRS_RCPT_DATE','NUM_REFERENCE_NO','POL_EXP_DATE', 'POL_INCEPT_DATE',
       'PREMIUM_AMOUNT_INR_WITHOUT_TAX','RECORD_TYPE_DESC').drop_duplicates()
# .filter(col('NUM_REFERENCE_NO').isin([202001010068276]))

In [ ]:
tp_claims_7 = tp_claims_6.join(prem_reg,[prem_reg.POLICY_NO==tp_claims_6.TXT_POLICY_NO_CHAR,
                                         prem_reg.POL_INCEPT_DATE==tp_claims_6.DAT_POLICY_STARTDATE],
                               how='left').drop_duplicates()

In [ ]:
tp_claims_7.columns

#### VEHICLE AGE, PROPOSAL TYPE

In [ ]:
gscPythonOptions = {
        "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456","dbschema": "public",
        "dbtable": "policy_dh_risk_headers_mot","partitions":16,"server.port": '1150-1170',"partitionColumn":"row_num"}
pol_head_mot = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('REFERENCE_NUM','PROPOSAL_TYPE','VEHICLE_AGE','RISK1').drop_duplicates()

In [ ]:
tp_claims_8 = tp_claims_7.join(pol_head_mot,[pol_head_mot.REFERENCE_NUM==tp_claims_7.NUM_REFERENCE_NO],
                               how='left').drop(pol_head_mot.REFERENCE_NUM).drop_duplicates()

In [ ]:
tp_claims_8.columns

In [ ]:
# tp_claims_8 = tp_claims_8.withColumn('POLICY_YEAR',sf.year(sf.to_timestamp('DAT_POLICY_STARTDATE', 'yyyy-mm-dd')))

In [ ]:
# tp_claims_8 = tp_claims_8.withColumn('POLICY_YEAR_BEFORE', ( tp_claims_8['POLICY_YEAR'] - 1))

### PREVIOUS POLICY DETAILS

In [ ]:
gscPythonOptions = {
        "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456","dbschema": "public",
        "dbtable": "policy_gc_previous_policy_details_tab","partitions":8,"server.port": '1150-1170',"partitionColumn":"row_num"}
prev_pol = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('NUM_REFERENCE_NUMBER','CORPORATE_CUSTOMER_ID').drop_duplicates()

In [ ]:
tp_claims_9 = tp_claims_8.join(prev_pol,[prev_pol.NUM_REFERENCE_NUMBER==tp_claims_8.NUM_REFERENCE_NO],how='left').drop(prev_pol.NUM_REFERENCE_NUMBER)

In [ ]:
tp_claims_9=tp_claims_9.drop_duplicates()

In [ ]:
tp_claims_9.columns

In [ ]:
tp_claims_9.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
output_index = 'claims_data_TP_20apr'

import sys

try:
    tp_claims_9.write.format("greenplum")\
    .option("dbtable",output_index)\
    .option('dbschema','analytics')\
    .option("server.port",'1150-1170')\
    .option("url","jdbc:postgresql://10.35.12.194:5432/gpadmin")\
    .option("user", "gpspark")\
    .option("password","spark@456")\
    .option("partitions",8)\
    .mode('overwrite')\
    .save()

except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
tp_claims_9.unpersist()

In [2]:
from pg import DB
db = DB(dbname="gpadmin", user='gpanalytics', passwd='ana@678', host='10.35.12.194', port= 5432)

In [ ]:
a = db.query("DELETE FROM analytics.claim_hist_24apr where num_claim_no in (select distinct gc.num_claim_no \
             from public.claim_gc_gc_clm_gen_info gc where extract(year from gc.dat_notification_date)>=2018 \
             and cast(gc.dat_update_date as date)=(CURRENT_DATE-INTERVAL '1 day'))")
print("Deleted records for modified claim numbers  : "+ a)

In [4]:
a = db.query("DROP TABLE analytics.claims_hist_29_jun")
# print("Deleted records for modified claim numbers  : "+ a)

ERROR:  table "claims_hist_29_jun" does not exist

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pg.py", line 1975, in query
    return self.db.query(command)
pg.ProgrammingError: ERROR:  table "claims_hist_29_jun" does not exist




In [ ]:
s = db.query("Insert into analytics.claim_hist_24apr (txt_master_claim_no,num_claim_no,dat_loss_date, \
             dat_notification_date,dat_update_date ,txt_policy_no_char ,featureno ,productname, txt_cover_description ,\
             cwpreason ,rpdreason, producer_cd,crs_rcpt_date,pol_incept_date,covernotenumber,cover_note_date ) \
             (select distinct t1.txt_master_claim_no ,t1.num_claim_no ,t1.dat_loss_date ,t1.dat_notification_date ,\
             t1.dat_update_date ,t1.txt_policy_no_char ,t1.featureno, t1.productname, t1.txt_cover_description, t1.cwpreason, \
             t1.rpdreason,t1.producer_cd,t1.crs_rcpt_date,t1.pol_incept_date,t1.covernotenumber ,t1.cover_note_date from \
             ((((select distinct gc.txt_master_claim_no,gc.num_claim_no,gc.dat_loss_date,gc.txt_policy_no_char, \
             gc.dat_policy_inception_date, gc.dat_notification_date,gc.dat_update_date,em.featureno, \
             pm.productname, rgc.txt_cover_description,gc.num_reference_no from public.claim_gc_gc_clm_gen_info gc, \
             (select distinct num_claim_no, featureno,injuryname from public.claim_dh_gc_clm_gen_info_extra_mot) em, \
             public.reference_gc_covercodemaster rgc, \
             (select productname,productcode from public.underwriting_gc_uw_product_master \
             where productcode in (3121,3122,3124,3125,3126,3183,3184,3186,3188,3189,3191)) pm \
             where gc.num_claim_no = em.num_claim_no and gc.num_nature_of_loss = rgc.num_cover_code and \
             gc.num_update_no = (select max(a.num_update_no) from public.claim_gc_gc_clm_gen_info a \
             where a.num_claim_no=gc.num_claim_no) and gc.num_product_code = pm.productcode) f1 \
             left join (select distinct g.num_claim_no as claim_no1, c.txt_repudiation_reason_desc as cwpReason, \
             c.txt_info1 as cwp from public.claim_gc_gc_clm_gen_info g,public.claim_gc_gc_clmmst_repudiation_reason c \
             where g.num_repudiation_reason_id in \
             (select num_repudiation_reason_id from public.claim_gc_gc_clmmst_repudiation_reason r \
              where r.txt_info1 in ('CWP')) and c.num_repudiation_reason_id=g.num_repudiation_reason_id \
              and g.num_update_no = (Select Max(b.num_update_no )From public.claim_gc_gc_clm_gen_info b \
              Where b.num_claim_no = g.num_claim_no and coalesce(g.num_repudiation_reason_id ,0) <> 0)) f2 \
             on f1.num_claim_no=f2.claim_no1) f3 left join \
             (select distinct g.num_claim_no as claim_no2,c.txt_repudiation_reason_desc as rpdreason,c.txt_info1 as rpd \
             from public.claim_gc_gc_clm_gen_info g,public.claim_gc_gc_clmmst_repudiation_reason c \
             where g.num_repudiation_reason_id in \
             (select num_repudiation_reason_id from public.claim_gc_gc_clmmst_repudiation_reason r \
             where r.txt_info1 in ('REPUDIATION')) and c.num_repudiation_reason_id =g.num_repudiation_reason_id \
             and g.num_update_no = (Select Max(b.num_update_no )From public.claim_gc_gc_clm_gen_info b \
             Where b.num_claim_no = g.num_claim_no and coalesce(g.num_repudiation_reason_id ,0) <> 0)) f4 \
             on f3.num_claim_no=f4.claim_no2) a \
             left join \
             (select distinct num_reference_no as ref_no,producer_cd, pol_incept_date, covernotenumber ,cover_note_date ,\
             crs_rcpt_date from registers.premium_register) pr on \
             a.num_reference_no=pr.ref_no)t1 where extract(year from t1.dat_notification_date)>=2018 and cast(t1.dat_update_date as date)=(CURRENT_DATE-INTERVAL '1 day'))")

print("Inserted records for modified claim numbers  : "+ s)

In [5]:
s = db.query("Create table analytics.claims_hist_29_jun As \
             (select distinct t1.txt_master_claim_no ,t1.num_claim_no ,t1.dat_loss_date ,t1.dat_notification_date ,\
             t1.dat_update_date ,t1.txt_policy_no_char ,t1.featureno, t1.productname, t1.txt_cover_description, t1.cwpreason, \
             t1.rpdreason,t1.producer_cd,t1.crs_rcpt_date,t1.pol_incept_date,t1.covernotenumber ,t1.cover_note_date from \
             ((((select distinct gc.txt_master_claim_no,gc.num_claim_no,gc.dat_loss_date,gc.txt_policy_no_char, \
             gc.dat_policy_inception_date, gc.dat_notification_date,gc.dat_update_date,em.featureno, \
             pm.productname, rgc.txt_cover_description,gc.num_reference_no from public.claim_gc_gc_clm_gen_info gc, \
             (select distinct num_claim_no, featureno,injuryname from public.claim_dh_gc_clm_gen_info_extra_mot) em, \
             public.reference_gc_covercodemaster rgc, \
             (select productname,productcode from public.underwriting_gc_uw_product_master \
             where productcode in (3121,3122,3124,3125,3126,3183,3184,3186,3188,3189,3191)) pm \
             where gc.num_claim_no = em.num_claim_no and gc.num_nature_of_loss = rgc.num_cover_code and \
             gc.num_update_no = (select max(a.num_update_no) from public.claim_gc_gc_clm_gen_info a \
             where a.num_claim_no=gc.num_claim_no) and gc.num_product_code = pm.productcode) f1 \
             left join (select distinct g.num_claim_no as claim_no1, c.txt_repudiation_reason_desc as cwpReason, \
             c.txt_info1 as cwp from public.claim_gc_gc_clm_gen_info g,public.claim_gc_gc_clmmst_repudiation_reason c \
             where g.num_repudiation_reason_id in \
             (select num_repudiation_reason_id from public.claim_gc_gc_clmmst_repudiation_reason r \
              where r.txt_info1 in ('CWP')) and c.num_repudiation_reason_id=g.num_repudiation_reason_id \
              and g.num_update_no = (Select Max(b.num_update_no )From public.claim_gc_gc_clm_gen_info b \
              Where b.num_claim_no = g.num_claim_no and coalesce(g.num_repudiation_reason_id ,0) <> 0)) f2 \
             on f1.num_claim_no=f2.claim_no1) f3 left join \
             (select distinct g.num_claim_no as claim_no2,c.txt_repudiation_reason_desc as rpdreason,c.txt_info1 as rpd \
             from public.claim_gc_gc_clm_gen_info g,public.claim_gc_gc_clmmst_repudiation_reason c \
             where g.num_repudiation_reason_id in \
             (select num_repudiation_reason_id from public.claim_gc_gc_clmmst_repudiation_reason r \
             where r.txt_info1 in ('REPUDIATION')) and c.num_repudiation_reason_id =g.num_repudiation_reason_id \
             and g.num_update_no = (Select Max(b.num_update_no )From public.claim_gc_gc_clm_gen_info b \
             Where b.num_claim_no = g.num_claim_no and coalesce(g.num_repudiation_reason_id ,0) <> 0)) f4 \
             on f3.num_claim_no=f4.claim_no2) a \
             left join \
             (select distinct num_reference_no as ref_no,producer_cd, pol_incept_date, covernotenumber ,cover_note_date ,\
             crs_rcpt_date from registers.premium_register) pr on \
             a.num_reference_no=pr.ref_no)t1 where extract(year from t1.dat_notification_date)>=2018 \
             and cast(t1.dat_update_date as date)<=(CURRENT_DATE-INTERVAL '1 day'))")

print("Inserted records for modified claim numbers  : "+ s)

Inserted records for modified claim numbers  : 3074026

In [6]:
spark.stop()